In [1]:
#!pip uninstall tensorflow
!pip install tensorflow==2.15
!pip install gym
!pip install keras
!pip install keras-rl2

Defaulting to user installation because normal site-packages is not writeable
  Using cached ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached tensorboard-2.15.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.7 MB/s eta 0:00:0031m12.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.0 MB/s eta 0:00:000m eta 0:00:01:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 14.7 MB/s eta 0:00:00m eta 0:00:010:01:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboar

In [18]:
import gym
import random

In [19]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

/home/giacomo/.local/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [20]:
states

4

In [21]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0, 1])
        print(env.step(action))
        n_state, reward, done, info, boh = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    

(array([ 0.00068803,  0.22757941, -0.01588403, -0.33797422], dtype=float32), 1.0, False, False, {})
(array([ 0.0136981 ,  0.6183541 , -0.03535599, -0.9353504 ], dtype=float32), 1.0, False, False, {})
(array([ 0.04234387,  1.0097077 , -0.0788416 , -1.5480477 ], dtype=float32), 1.0, False, False, {})
(array([ 0.08665168,  1.0119214 , -0.1470876 , -1.6075779 ], dtype=float32), 1.0, False, False, {})
(array([ 0.12326635,  0.62633044, -0.20652176, -1.1324474 ], dtype=float32), 1.0, False, False, {})
Episode:1 Score:5.0
(array([-0.00771558,  0.22738522,  0.00174585, -0.33921972], dtype=float32), 1.0, False, False, {})
(array([ 0.00528177,  0.6176742 , -0.01766557, -0.92561704], dtype=float32), 1.0, False, False, {})
(array([ 0.03389586,  0.6183924 , -0.06065389, -0.9426671 ], dtype=float32), 1.0, False, False, {})
(array([ 0.05474646,  0.23020613, -0.09290008, -0.4030143 ], dtype=float32), 1.0, False, False, {})
(array([ 0.06008091,  0.23292823, -0.10378048, -0.46375528], dtype=float32), 1.0

/home/giacomo/.local/lib/python3.10/site-packages/gym/envs/classic_control/cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(
/home/giacomo/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/home/giacomo/.local/lib/python3.10/site-packages/gym/envs/classic_control/cartpole.py:177: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


In [22]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [31]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [32]:
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__

In [33]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

model = build_model(states, actions)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 4)                 0         
                                                                 
 dense_6 (Dense)             (None, 24)                120       
                                                                 
 dense_7 (Dense)             (None, 24)                600       
                                                                 
 dense_8 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770 (3.01 KB)
Trainable params: 770 (3.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [38]:
dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

2024-06-27 12:30:24.847307: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_8_3/bias/Assign' id:1746 op device:{requested: '', assigned: ''} def:{{{node dense_8_3/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_8_3/bias, dense_8_3/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected flatten_2_input to have shape (1, 4) but got array with shape (1, 2)